In [1]:
import numpy as np
import qiskit.pulse as pulse
from qiskit_dynamics.pulse.pulse_simulator import PulseSimulator
from qiskit_dynamics import Solver

In [2]:
Z = np.array([[-1., 0.], [0., 1.]])
X = np.array([[0., 1.], [1., 0.]])

r = 0.1

static_ham = 2 * np.pi * 5 * Z / 2
drive_op = 2 * np.pi * r * X / 2

solver = Solver(
    static_hamiltonian=static_ham,
    hamiltonian_operators=[drive_op],
    hamiltonian_channels=['d0'],
    channel_carrier_freqs={'d0': 5.},
    dt=0.1,
    rotating_frame=static_ham
)

In [3]:
backend = PulseSimulator(solver=solver, subsystem_dims=[2])

In [ ]:
from qiskit.pulse import library

amp = 1
sigma = 10
num_samples = 64
#%%
gauss = pulse.library.Gaussian(num_samples, amp, sigma,
                              name="Parametric Gauss")
gauss.draw()


with pulse.build() as schedule:
    # note: carrier frequency is automatically set to channel_carrier_freq if nothing
    # specified in schedule. Is this what we want?
    # This is baked into the InstructionToSchedule object. What does it do on the backends?
    #pulse.set_frequency(5., pulse.DriveChannel(0))
    pulse.play(gauss, pulse.DriveChannel(0))

result = backend.run(schedule, solver_options={'atol': 1e-8, 'rtol': 1e-8})
#%%
result[0].y[-1]

In [5]:
area = gauss.get_waveform().samples.sum() * 0.1
area

(2.48630065066594+1.5159645885691881e-16j)

In [19]:
from scipy.linalg import expm

expm(-1j * 2 * np.pi * area * r * X / 4)

array([[9.24700885e-01-9.06536913e-18j, 2.20196396e-17-3.80694462e-01j],
       [2.20196396e-17-3.80694462e-01j, 9.24700885e-01-9.06536913e-18j]])

In [7]:
from qiskit_dynamics import Solver
import numpy as np

In [22]:
Z = np.array([[1., 0.], [0., -1.]])
X = np.array([[0., 1.], [1., 0.]])

test_solver = Solver(
    static_hamiltonian=2 * X + 3 * Z,
    hamiltonian_operators=[X],
    rotating_frame=Z + X
)

In [23]:
test_solver.model.static_operator

Array([[ 2.+0.j,  1.+0.j],
       [ 1.+0.j, -2.+0.j]], backend='numpy')

In [24]:
test_solver.model.operators

Array([[[ 2.96506192e-17+0.j,  1.00000000e+00+0.j],
        [ 1.00000000e+00+0.j, -7.71461485e-17+0.j]]], backend='numpy')